In [1]:
import warnings
import requests
import base64
import time
import math
import copy
import json
import os
import io
import gc

from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, BitsAndBytesConfig, AutoProcessor, PaliGemmaForConditionalGeneration, GPTQConfig
from exllamav2 import ExLlamaV2, ExLlamaV2Config, ExLlamaV2Cache, ExLlamaV2Tokenizer
from exllamav2.generator import ExLlamaV2Sampler, ExLlamaV2DynamicGenerator, ExLlamaV2BaseGenerator
from flask import Flask, render_template, request, jsonify
from scipy.special import softmax
from typing import Union, List
from llama_cpp import Llama
from tqdm import tqdm
from PIL import Image
import bitsandbytes
import numpy as np
import flash_attn
import random
import torch

/home/tobi/miniconda3/envs/assba/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/tobi/miniconda3/envs/assba/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [39]:
def use_llm(task_input, system_instruct=None, forced_beginning=None, directly_get_reply=True):
    inputs = {}
    
    inputs['chat'] = []
    if system_instruct != None and system_instruct.strip() != "":
        inputs['chat'].append({"role": "system", "content": system_instruct})
    inputs['chat'].append({"role": "User", "content": task_input})
    if forced_beginning != None and forced_beginning.strip() != "":
        inputs['chat'].append({"role": "AI", "content": forced_beginning})
        
    inputs['model'] = "gemma-2-9b"
    inputs['model_dtype'] = "bfloat16"
    inputs['max_new_tokens'] = "512"
    inputs['debugmode'] = True

    
    url = f"http://127.0.0.1:10000/infer"
    response = requests.post(url, json=inputs) 
    if response.json()['status'] == "error":
        print(response.json()['error-info'])
        return json.dumps(response.json())
    ai_reply = response.json()['returned_content'][0]
    info = response.json()['info']
    
    returned_content = [
        {'role': 'AI', 'content': "\n".join(ai_reply)}
    ]
    if directly_get_reply:
        return returned_content[-1]['content']
    else:
        return json.dumps({'status': 'success', 'returned_content': returned_content, 'info': info})

In [40]:
query = """Rewrite the following specific task into a general, symbolized task description.
Use placeholders for names, numbers, and items.
Example:
User:
Peter has 5 apples and 7 bananas. If he eats 3 apples and trades one apple for a banana, what is he left with?

Answer:
{
    "symbolized task": "[main object] has [number 1] [item 1] and [number 2] [item 2].
[number 3] of [item 1] are removed.
[number 4] of [item 1] is changed into [number 5] of [item 2].
What are the values of [item 1] and [item 2]?"
}

User:
Peter has 5 candles that are all the same length. He lights them all at the same time. After a while, he blows out the candles one after the other. Which of the five candles was the first one he has blown out?
Here is a figure of the five candles after they have been blown out. The number of = represents the length of the candle. Respond with the label of the candle that has been blown out first by Peter.
1) ====
2) =======
3) ========
4) =
5) ==

Answer:
"""

In [43]:
print(use_llm(
    task_input = query,
    forced_beginning = "{\n    \"symbolized task\": "
))

"Peter has [number] [item] that are all the same [attribute]. 
    Peter lights them all at the same time. 
    After a while, he blows out one [item] after the other. 
    Which [item] was the first one he blew out?
    Here is a figure of the five [item] after they have been blown out. 
    The number of [item] represents the length of the [item]. 
    Respond with the label of the [item] that was first blown out by Peter."
} 



Let me know if you'd like to try another one!


In [43]:
print(use_llm(
    task_description = "You will be given a string. decide if it is an english noun / pronoun (\"task_solution\": true) or not (\"task_solution\": false). reply in json format.", 
    task_input = "\"narr\"",
    forced_beginning = "{\n    \"task_solution\": "
))

false
}
